Part 2

In [3]:
import pandas as pd

# Load the final processed dataset
df = pd.read_csv("nyc_taxi_final.csv")

# Check the first few rows
print(df.head())

   vendorid tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2019-01-01 00:46:40   2019-01-01 00:53:20              1.0   
1         1  2019-01-01 00:59:47   2019-01-01 01:18:59              1.0   
2         1  2019-01-01 00:21:28   2019-01-01 00:28:37              1.0   
3         1  2019-01-01 00:32:01   2019-01-01 00:45:39              1.0   
4         1  2019-01-01 00:57:32   2019-01-01 01:09:32              2.0   

   trip_distance  ratecodeid store_and_fwd_flag  pulocationid  dolocationid  \
0      -0.441643           1                  N         151.0         239.0   
1      -0.082501           1                  N         239.0         246.0   
2      -0.506941           1                  N         163.0         229.0   
3       0.276640           1                  N         229.0           7.0   
4      -0.245747           1                  N         141.0         234.0   

   payment_type  ...  pickup_hour  pickup_day  pickup_weekday  pickup_mont

In [ ]:
df.dtypes

vendorid                   int64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count          float64
trip_distance            float64
ratecodeid                 int64
store_and_fwd_flag        object
pulocationid             float64
dolocationid             float64
payment_type               int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
pickup_hour                int64
pickup_day                 int64
pickup_weekday             int64
pickup_month               int64
trip_duration            float64
speed_mph                float64
is_weekend                 int64
is_rush_hour               int64
is_night                   int64
high_fare                  int64
dtype: object

: 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KDTree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Efficient kNN class using KDTree
class KNNFast:
    def __init__(self, k=3):
        self.k = k
        self.tree = None
        self.y_train = None

    def fit(self, X_train, y_train):
        self.tree = KDTree(X_train)
        self.y_train = y_train

    def predict(self, X_test):
        distances, indices = self.tree.query(X_test, k=self.k)
        predictions = []
        for i in range(len(X_test)):
            k_labels = self.y_train[indices[i]]
            most_common = np.bincount(k_labels).argmax()
            predictions.append(most_common)
        return np.array(predictions)

# Feature matrix and target
X = df[['trip_distance', 'fare_amount', 'total_amount', 'tolls_amount',
        'pickup_hour', 'pickup_day', 'pickup_weekday', 'pickup_month',
        'trip_duration', 'speed_mph', 'is_weekend', 'is_rush_hour', 'is_night',
        'pulocationid', 'passenger_count', 'payment_type', 'improvement_surcharge',
        'tip_amount', 'mta_tax', 'extra']].values
y = df['high_fare'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate fast kNN
knn_fast = KNNFast(k=3)
knn_fast.fit(X_train, y_train)
y_pred_knn = knn_fast.predict(X_test)
print("Fast kNN Accuracy:", accuracy_score(y_test, y_pred_knn))


Fast kNN Accuracy: 0.9662676822633297


: 

In [ ]:
from sklearn.linear_model import LogisticRegression

# Logistic Regression Model
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

# Evaluate performance
from sklearn.metrics import classification_report
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))


Logistic Regression Accuracy: 0.9978237214363439
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       986
           1       1.00      1.00      1.00       852

    accuracy                           1.00      1838
   macro avg       1.00      1.00      1.00      1838
weighted avg       1.00      1.00      1.00      1838



c:\Users\crist\miniconda3\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


: 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=5, min_samples_leaf=4, random_state=42)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

# Evaluate performance
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

Random Forest Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       986
           1       1.00      1.00      1.00       852

    accuracy                           1.00      1838
   macro avg       1.00      1.00      1.00      1838
weighted avg       1.00      1.00      1.00      1838



: 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Gradient Boosting Model
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)

# Evaluate performance
print("Gradient Boosting Accuracy:", accuracy_score(y_test, y_pred_gb))
print(classification_report(y_test, y_pred_gb))

Gradient Boosting Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       986
           1       1.00      1.00      1.00       852

    accuracy                           1.00      1838
   macro avg       1.00      1.00      1.00      1838
weighted avg       1.00      1.00      1.00      1838



: 

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Deep Neural Network model with 10 layers
model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),  # Layer 1
    layers.Dense(128, activation='relu'),  # Layer 2
    layers.Dense(64, activation='relu'),   # Layer 3
    layers.Dense(64, activation='relu'),   # Layer 4
    layers.Dense(32, activation='relu'),   # Layer 5
    layers.Dense(32, activation='relu'),   # Layer 6
    layers.Dense(16, activation='relu'),   # Layer 7
    layers.Dense(16, activation='relu'),   # Layer 8
    layers.Dense(8, activation='relu'),    # Layer 9
    layers.Dense(1, activation='sigmoid')  # Layer 10 (Output)
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Deep Learning Model Accuracy: {test_acc}")


Epoch 1/20
230/230 [==============================] - 2s 5ms/step - loss: 0.4857 - accuracy: 0.8143
Epoch 2/20
230/230 [==============================] - 1s 4ms/step - loss: 0.3057 - accuracy: 0.8780
Epoch 3/20
230/230 [==============================] - 1s 4ms/step - loss: 0.1165 - accuracy: 0.9592
Epoch 4/20
230/230 [==============================] - 1s 4ms/step - loss: 0.0945 - accuracy: 0.9653
Epoch 5/20
230/230 [==============================] - 1s 4ms/step - loss: 0.1233 - accuracy: 0.9737
Epoch 6/20
230/230 [==============================] - 1s 3ms/step - loss: 0.0639 - accuracy: 0.9759
Epoch 7/20
230/230 [==============================] - 1s 3ms/step - loss: 0.0650 - accuracy: 0.9742
Epoch 8/20
230/230 [==============================] - 1s 3ms/step - loss: 0.0511 - accuracy: 0.9784
Epoch 9/20
230/230 [==============================] - 1s 4ms/step - loss: 0.0534 - accuracy: 0.9771
Epoch 10/20
230/230 [==============================] - 1s 3ms/step - loss: 0.0482 - accuracy: 0.9785

: 

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, adjusted_rand_score
import matplotlib.pyplot as plt

# K-Means Clustering with 2 clusters
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
kmeans.fit(X)

# Cluster assignments
labels = kmeans.labels_

# Compute Silhouette Score
silhouette = silhouette_score(X, labels)
print(f"Silhouette Score: {silhouette}")

# Plot the results
plt.scatter(X[:, 0], X[:, 1], c=kmeans.labels_, cmap='viridis')
plt.title("K-Means Clustering")
plt.show()

NameError: name 'X' is not defined

: 

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, adjusted_rand_score
import numpy as np
import matplotlib.pyplot as plt

# DBSCAN clustering
dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan_labels = dbscan.fit_predict(X)

# Count number of clusters (excluding noise points labeled as -1)
num_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
num_noise_points = np.sum(dbscan_labels == -1)

print(f"Number of clusters found by DBSCAN: {num_clusters}")
print(f"Number of noise points: {num_noise_points}")

# Compute Silhouette Score (only if there are more than 1 clusters)
if num_clusters > 1:
    silhouette = silhouette_score(X, dbscan_labels)
    print(f"Silhouette Score: {silhouette}")
else:
    silhouette = None
    print("Silhouette Score not applicable (only one cluster found).")

# Plot the results
plt.scatter(X[:, 0], X[:, 1], c=dbscan_labels, cmap='viridis')
plt.title("DBSCAN Clustering")
plt.show()


NameError: name 'X' is not defined

: 

In [ ]:
# Comparison table
models = ['kNN', 'Logistic Regression', 'Random Forest', 'Gradient Boosting', 'Deep Learning', 'K-Means', 'DBSCAN']
accuracies = [
    accuracy_score(y_test, y_pred_knn),  # kNN
    accuracy_score(y_test, y_pred_lr),   # Logistic Regression
    accuracy_score(y_test, y_pred_rf),   # Random Forest
    accuracy_score(y_test, y_pred_gb),   # Gradient Boosting
    test_acc,                            # Deep Learning
    np.nan,                              # K-Means does not have accuracy
    np.nan                               # DBSCAN does not have accuracy
]

model_comparison = pd.DataFrame({
    'Model': models,
    'Accuracy': accuracies
})

print(model_comparison)

                 Model  Accuracy
0                  kNN  0.966268
1  Logistic Regression  0.997824
2        Random Forest  1.000000
3    Gradient Boosting  1.000000
4        Deep Learning  0.988030
5              K-Means       NaN
6               DBSCAN       NaN


: 